In [2]:
import pandas as pd

# Path file input dan output
input_path = r"C:\Users\ghiff\Documents\kuliah\Analisa_Sentimen_Gojek\percobaan 16\driver.csv"
output_path = r"C:\Users\ghiff\Documents\kuliah\Analisa_Sentimen_Gojek\percobaan 17\driver_labeled.csv"

# Load dataset
data = pd.read_csv(input_path)

# Label categories
label_categories = [
    ("Permasalahan Sulit Mendapatkan Driver", ["susah", "cari", "nunggu", "tunggu"]),
    ("Permasalahan Driver Lama Datang", ["lama", "telat", "nunggu", "waktu"]),
    ("Permasalahan Driver Tidak Profesional", ["kecewa", "kurang", "layanan"]),
    ("Permasalahan Driver Membatalkan Order", ["cancel", "batal", "drivernya"]),
    ("Permasalahan Lokasi Penjemputan", ["jemput", "lokasi", "titik", "jalan"]),
    ("Permasalahan Cuaca dan Kondisi Jalan", ["hujan", "macet", "jalan"]),
    ("Permasalahan Biaya Tidak Sesuai", ["biaya", "mahal", "harga"]),
    ("Permasalahan Driver Tidak Ramah", ["kurang", "ramah", "kecewa"]),
    ("Permasalahan Driver Tidak Datang", ["tidak", "datang", "nunggu"]),
    ("Permasalahan Driver Tidak Menemukan Lokasi", ["cari", "lokasi", "salah"]),
    ("Permasalahan Driver Lambat Merespons", ["lama", "respon", "tidak"]),
    ("Permasalahan Driver Mengubah Rute", ["jalan", "rute", "salah"]),
    ("Permasalahan Driver Meminta Pembatalan", ["minta", "cancel", "batal"]),
    ("Permasalahan Driver Tidak Menggunakan Aplikasi", ["tidak", "guna", "aplikasi"]),
    ("Permasalahan Driver Menagih Biaya Tambahan", ["biaya", "lebih", "bayar"]),
    ("Permasalahan Driver Tidak Sesuai Pesanan", ["tidak", "pesan", "drivernya"]),
    ("Permasalahan Driver Datang ke Lokasi Salah", ["salah", "lokasi", "jemput"]),
    ("Permasalahan Driver Tidak Membantu", ["tidak", "tolong", "kurang"]),
    ("Permasalahan Driver Membatalkan Pesanan Tanpa Alasan", ["cancel", "batal", "alasan"]),
    ("Permasalahan Driver Tidak Menyediakan Masker atau Keamanan", ["tidak", "masker", "aman"])
]

# Fungsi untuk memberikan label berdasarkan kategori
def assign_label(tokens):
    for label, keywords in label_categories:
        if any(keyword in tokens for keyword in keywords):
            return label
    return "Tidak Ada Permasalahan"

# Terapkan fungsi label pada dataset
data['permasalahan'] = data['stemmed_tokens'].apply(lambda x: assign_label(eval(x)))

# Simpan hasil ke CSV baru
data.to_csv(output_path, index=False)

output_path


'C:\\Users\\ghiff\\Documents\\kuliah\\Analisa_Sentimen_Gojek\\percobaan 17\\driver_labeled.csv'

In [4]:
import pandas as pd

# Load the CSV file
file_path = r"C:\Users\ghiff\Documents\kuliah\Analisa_Sentimen_Gojek\percobaan 17\driver_labeled.csv"
data = pd.read_csv(file_path)

# Group and count occurrences of each issue in the "permasalahan" column
issue_counts = data['permasalahan'].value_counts()

# Display the unique issues and their counts
issue_counts


permasalahan
Permasalahan Sulit Mendapatkan Driver                         3453
Tidak Ada Permasalahan                                        3386
Permasalahan Driver Membatalkan Order                         1206
Permasalahan Driver Tidak Profesional                          820
Permasalahan Driver Tidak Menggunakan Aplikasi                 804
Permasalahan Lokasi Penjemputan                                785
Permasalahan Driver Tidak Ramah                                650
Permasalahan Biaya Tidak Sesuai                                581
Permasalahan Driver Lama Datang                                558
Permasalahan Driver Tidak Sesuai Pesanan                       293
Permasalahan Driver Tidak Membantu                             280
Permasalahan Cuaca dan Kondisi Jalan                           185
Permasalahan Driver Meminta Pembatalan                         127
Permasalahan Driver Menagih Biaya Tambahan                     109
Permasalahan Driver Tidak Menemukan Lokasi       

In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE

# Load dataset
path = r"C:\Users\ghiff\Documents\kuliah\Analisa_Sentimen_Gojek\percobaan 17\driver_labeled.csv"
data = pd.read_csv(path)

# Ekstraksi fitur (TF-IDF)
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(data['stemmed_tokens'].apply(lambda x: ' '.join(eval(x)))).toarray()

# Label (Target)
y = data['permasalahan']

# Split data sebelum resampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Mengatasi ketidakseimbangan data dengan SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Cek distribusi kelas setelah resampling
print("Distribusi kelas setelah oversampling dengan SMOTE:")
print(pd.Series(y_train_resampled).value_counts())

# Model Naive Bayes
model = MultinomialNB()
model.fit(X_train_resampled, y_train_resampled)

# Evaluasi
y_pred = model.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Simpan model dan TF-IDF vectorizer
import pickle

with open("naive_bayes_model.pkl", "wb") as model_file:
    pickle.dump(model, model_file)

with open("tfidf_vectorizer.pkl", "wb") as tfidf_file:
    pickle.dump(tfidf, tfidf_file)

print("Model dan TF-IDF vectorizer berhasil disimpan.")


c:\Users\ghiff\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\ghiff\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


Distribusi kelas setelah oversampling dengan SMOTE:
permasalahan
Permasalahan Sulit Mendapatkan Driver                         2763
Permasalahan Driver Lama Datang                               2763
Permasalahan Driver Mengubah Rute                             2763
Permasalahan Driver Tidak Datang                              2763
Permasalahan Driver Tidak Menemukan Lokasi                    2763
Permasalahan Driver Lambat Merespons                          2763
Permasalahan Driver Meminta Pembatalan                        2763
Permasalahan Cuaca dan Kondisi Jalan                          2763
Permasalahan Driver Menagih Biaya Tambahan                    2763
Permasalahan Biaya Tidak Sesuai                               2763
Permasalahan Driver Tidak Membantu                            2763
Permasalahan Driver Tidak Ramah                               2763
Permasalahan Driver Tidak Sesuai Pesanan                      2763
Permasalahan Driver Tidak Menggunakan Aplikasi                27

In [17]:
import joblib

# Load model dan vectorizer
model = joblib.load("naive_bayes_model.pkl")
tfidf = joblib.load("tfidf_vectorizer.pkl")

# Input baru
new_input = ["drivernya telat"]
new_input_transformed = tfidf.transform(new_input).toarray()

# Prediksi
prediction = model.predict(new_input_transformed)
print(f"Prediksi Permasalahan: {prediction[0]}")


Prediksi Permasalahan: Permasalahan Driver Lama Datang


In [18]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import pickle

# Load dataset
path = r"C:\Users\ghiff\Documents\kuliah\Analisa_Sentimen_Gojek\percobaan 17\driver_labeled.csv"
data = pd.read_csv(path)

# Filter data: hapus data dengan score == 5
data = data[data['score'] != 5]

# Ekstraksi fitur (TF-IDF)
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(data['stemmed_tokens'].apply(lambda x: ' '.join(eval(x)))).toarray()

# Label (Target)
y = data['permasalahan']

# Split data sebelum resampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Mengatasi ketidakseimbangan data dengan SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Cek distribusi kelas setelah resampling
print("Distribusi kelas setelah oversampling dengan SMOTE:")
print(pd.Series(y_train_resampled).value_counts())

# Model Naive Bayes
model = MultinomialNB()
model.fit(X_train_resampled, y_train_resampled)

# Evaluasi
y_pred = model.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Simpan model dan TF-IDF vectorizer
with open("naive_bayes_model_filtered.pkl", "wb") as model_file:
    pickle.dump(model, model_file)

with open("tfidf_vectorizer_filtered.pkl", "wb") as tfidf_file:
    pickle.dump(tfidf, tfidf_file)

print("Model dan TF-IDF vectorizer berhasil disimpan.")


Distribusi kelas setelah oversampling dengan SMOTE:
permasalahan
Permasalahan Driver Tidak Menggunakan Aplikasi                2500
Permasalahan Driver Tidak Profesional                         2500
Permasalahan Driver Mengubah Rute                             2500
Permasalahan Driver Tidak Datang                              2500
Permasalahan Cuaca dan Kondisi Jalan                          2500
Permasalahan Driver Tidak Ramah                               2500
Permasalahan Driver Tidak Membantu                            2500
Permasalahan Driver Lambat Merespons                          2500
Permasalahan Driver Meminta Pembatalan                        2500
Permasalahan Driver Menagih Biaya Tambahan                    2500
Tidak Ada Permasalahan                                        2500
Permasalahan Driver Tidak Sesuai Pesanan                      2500
Permasalahan Driver Tidak Menemukan Lokasi                    2500
Permasalahan Driver Lama Datang                               25

In [1]:
import joblib

# Load model dan vectorizer
model = joblib.load("naive_bayes_model_filtered.pkl")
tfidf = joblib.load("tfidf_vectorizer_filtered.pkl")

# Input baru
new_input = ["driver tidak mau membantu"]
new_input_transformed = tfidf.transform(new_input).toarray()

# Prediksi
prediction = model.predict(new_input_transformed)
print(f"Prediksi Permasalahan: {prediction[0]}")


Prediksi Permasalahan: Permasalahan Driver Tidak Datang
